In [38]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> Updating lists func:

    get_participants(event_link)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)


# Main( )

In [39]:
data_loc = '../data/'

In [40]:
fighters = pd.read_csv(data_loc + 'update_list_fighters.csv')
fighters.tail(3)

,fighter,fighter_url
690,Sijara Eubanks,http://ufcstats.com/fighter-details/4234f0097c...
691,Andre Ewell,http://ufcstats.com/fighter-details/5f6f049d54...
692,Matt Sayles,http://ufcstats.com/fighter-details/5b8ba8f551...


#### Extracting website data to data frame:

In [41]:
data = []
column_names = ['name','wins','losses','draws',
                'height','weight','reach','stance','dob',
                'slpm','str_acc','sapm','str_def','td_avg','td_acc','td_def','sub_avg',
                'last_update','last_match','url']

for url in tqdm(fighters.fighter_url):
    try:
        data.append(get_fighter_data(url=url))
    except:
        print('error on ' + url)
        continue

df = pd.DataFrame(data=data, columns=column_names)  
df.tail(3)

100%|██████████| 693/693 [06:18<00:00,  1.83it/s]


,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
690,Sijara Eubanks,8,7,0,"5'4""",125lbs.,"67""",Orthodox,"Apr27,1985",4.30,43%,3.43,57%,1.90,44%,66%,0.3,04-18-2023,12-18-2021,http://ufcstats.com/fighter-details/4234f0097c...
691,Andre Ewell,17,9,0,"5'8""",145lbs.,"75""",Southpaw,"Jan21,1988",4.28,40%,4.87,51%,0.36,75%,72%,0.1,04-18-2023,12-18-2021,http://ufcstats.com/fighter-details/5f6f049d54...
692,Matt Sayles,8,4,0,"5'7""",155lbs.,"68""",Orthodox,"Feb21,1994",4.83,41%,2.81,61%,0.36,16%,81%,0.7,04-18-2023,12-18-2021,http://ufcstats.com/fighter-details/5b8ba8f551...


#### Merge data with existing file: 

In [47]:
org = pd.read_csv(data_loc + 'ufc_fighters_stats.csv')
org.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
2370,David Levicki,1,3,0,"6'5""",275lbs.,--,NaN,--,0.0,0%,0.0,0%,0.0,0%,0%,0.0,04-18-2023,03-11-1994,http://ufcstats.com/fighter-details/49590e0508...
2371,Ray Wizard,0,1,0,--,--,--,NaN,--,0.0,0%,0.0,0%,0.0,0%,0%,0.0,04-18-2023,03-11-1994,http://ufcstats.com/fighter-details/ea0ad15545...
2372,Sean Daugherty,0,2,0,"6'0""",175lbs.,--,NaN,"Dec04,1975",0.0,0%,0.0,0%,0.0,0%,0%,0.0,04-18-2023,03-11-1994,http://ufcstats.com/fighter-details/a683f9ddb7...


In [48]:
# drop rows that needs update from org
mask = ~org['name'].isin(df.name)
org = org[mask].reset_index(drop=True)
org.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
1677,David Levicki,1,3,0,"6'5""",275lbs.,--,NaN,--,0.0,0%,0.0,0%,0.0,0%,0%,0.0,04-18-2023,03-11-1994,http://ufcstats.com/fighter-details/49590e0508...
1678,Ray Wizard,0,1,0,--,--,--,NaN,--,0.0,0%,0.0,0%,0.0,0%,0%,0.0,04-18-2023,03-11-1994,http://ufcstats.com/fighter-details/ea0ad15545...
1679,Sean Daugherty,0,2,0,"6'0""",175lbs.,--,NaN,"Dec04,1975",0.0,0%,0.0,0%,0.0,0%,0%,0.0,04-18-2023,03-11-1994,http://ufcstats.com/fighter-details/a683f9ddb7...


In [50]:
# combine original data frame with new one
data = pd.concat([org, df]).reset_index(drop=True) 
data.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
2370,Sijara Eubanks,8,7,0,"5'4""",125lbs.,"67""",Orthodox,"Apr27,1985",4.30,43%,3.43,57%,1.90,44%,66%,0.3,04-18-2023,12-18-2021,http://ufcstats.com/fighter-details/4234f0097c...
2371,Andre Ewell,17,9,0,"5'8""",145lbs.,"75""",Southpaw,"Jan21,1988",4.28,40%,4.87,51%,0.36,75%,72%,0.1,04-18-2023,12-18-2021,http://ufcstats.com/fighter-details/5f6f049d54...
2372,Matt Sayles,8,4,0,"5'7""",155lbs.,"68""",Orthodox,"Feb21,1994",4.83,41%,2.81,61%,0.36,16%,81%,0.7,04-18-2023,12-18-2021,http://ufcstats.com/fighter-details/5b8ba8f551...


#### Saving file:

In [51]:
data.to_csv(data_loc + 'ufc_fighters_stats.csv', index=0)
data.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
2370,Sijara Eubanks,8,7,0,"5'4""",125lbs.,"67""",Orthodox,"Apr27,1985",4.30,43%,3.43,57%,1.90,44%,66%,0.3,04-18-2023,12-18-2021,http://ufcstats.com/fighter-details/4234f0097c...
2371,Andre Ewell,17,9,0,"5'8""",145lbs.,"75""",Southpaw,"Jan21,1988",4.28,40%,4.87,51%,0.36,75%,72%,0.1,04-18-2023,12-18-2021,http://ufcstats.com/fighter-details/5f6f049d54...
2372,Matt Sayles,8,4,0,"5'7""",155lbs.,"68""",Orthodox,"Feb21,1994",4.83,41%,2.81,61%,0.36,16%,81%,0.7,04-18-2023,12-18-2021,http://ufcstats.com/fighter-details/5b8ba8f551...


### >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>